<a href="https://colab.research.google.com/github/MdNiamul/DataHioding/blob/main/pvd_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!/usr/bin/env python3
"""
Advanced Bengali Steganography Suite v3.0 - Military Grade Edition
================================================================

Advanced steganography system with:
- Military Grade AES-256-GCM encryption with intelligent auto-compression
- SHA-256 message integrity verification
- Real-time capacity monitoring with intelligent warnings
- Enhanced Bengali text support with UTF-8 optimization
- Pixel Value Differencing (PVD) steganography with improved algorithms
- Interactive GUI with modern design and colorful buttons
"""

import io
import base64
import math
import json
import zlib
import secrets
import hashlib
import time
from typing import Dict, Tuple, Union, Optional, List, Any
from PIL import Image
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Import cryptography components with error handling
try:
    from cryptography.hazmat.primitives.ciphers.aead import AESGCM
    from cryptography.hazmat.primitives import hashes
    from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
    from cryptography.hazmat.backends import default_backend
    CRYPTO_AVAILABLE = True
except ImportError:
    CRYPTO_AVAILABLE = False
    print("⚠️ Cryptography library not available. Install with: pip install cryptography")

# ============================================================================
# CORE ENCRYPTION AND COMPRESSION ENGINE
# ============================================================================

class MilitaryAESGCM:
    """
    Military Grade AES-256-GCM encryption with intelligent compression and SHA-256 integrity
    """

    VERSION = "3.0.0"
    ALGORITHM = "AES-256-GCM"
    DEFAULT_ITERATIONS = 100000
    SALT_SIZE = 16
    NONCE_SIZE = 12
    KEY_SIZE = 32

    @classmethod
    def analyze_compression(cls, data: bytes) -> Dict[str, Any]:
        """
        Intelligent compression analysis with multiple levels
        """
        if len(data) < 100:
            return {
                'recommended': False,
                'best_level': 0,
                'savings': 0,
                'reason': 'Data too small for effective compression'
            }

        # Test multiple compression levels
        test_levels = [1, 6, 9]
        best_level = 0
        best_ratio = 1.0
        results = {}

        for level in test_levels:
            try:
                compressed = zlib.compress(data, level=level)
                ratio = len(compressed) / len(data)
                savings = (1 - ratio) * 100

                results[level] = {
                    'compressed_size': len(compressed),
                    'ratio': ratio,
                    'savings': savings
                }

                # Must save at least 5% to be worthwhile
                if ratio < best_ratio and savings >= 5.0:
                    best_ratio = ratio
                    best_level = level

            except Exception:
                results[level] = {'error': True}

        return {
            'recommended': best_level > 0,
            'best_level': best_level,
            'best_ratio': best_ratio,
            'savings': (1 - best_ratio) * 100 if best_level > 0 else 0,
            'results': results,
            'reason': 'Beneficial compression found' if best_level > 0 else 'No significant compression benefit'
        }

    @classmethod
    def derive_key(cls, password: str, salt: bytes, iterations: int = None) -> bytes:
        """
        Derive encryption key using PBKDF2-HMAC-SHA256
        """
        if not isinstance(password, str) or len(password) < 8:
            raise ValueError("Password must be a string with minimum 8 characters")

        if len(salt) != cls.SALT_SIZE:
            raise ValueError(f"Salt must be exactly {cls.SALT_SIZE} bytes")

        iterations = iterations or cls.DEFAULT_ITERATIONS

        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=cls.KEY_SIZE,
            salt=salt,
            iterations=iterations,
            backend=default_backend()
        )

        return kdf.derive(password.encode('utf-8'))

    @classmethod
    def encrypt(cls, plaintext: bytes, password: str, auto_compress: bool = True) -> bytes:
        """
        Encrypt data with optional intelligent compression
        """
        if not isinstance(plaintext, bytes) or len(plaintext) == 0:
            raise ValueError("Plaintext must be non-empty bytes")

        # Step 1: Calculate SHA-256 hash for integrity
        message_hash = hashlib.sha256(plaintext).hexdigest()
        print(f"🔍 Message SHA-256: {message_hash[:16]}...{message_hash[-8:]}")

        # Step 2: Intelligent compression analysis
        payload_bytes = plaintext
        compression_info = cls.analyze_compression(plaintext) if auto_compress else {'recommended': False}
        compressed = False
        compression_level = 0

        if compression_info['recommended']:
            try:
                compressed_data = zlib.compress(plaintext, level=compression_info['best_level'])
                payload_bytes = compressed_data
                compressed = True
                compression_level = compression_info['best_level']
                print(f"🗜️ Smart compression: Level {compression_level}, "
                      f"{compression_info['savings']:.1f}% reduction")
            except Exception as e:
                print(f"⚠️ Compression failed, using original: {e}")
                compression_info['recommended'] = False

        if not compression_info['recommended']:
            print("📊 Compression analysis: Not beneficial, keeping original")

        # Step 3: Generate cryptographic parameters
        salt = secrets.token_bytes(cls.SALT_SIZE)
        nonce = secrets.token_bytes(cls.NONCE_SIZE)
        iterations = cls.DEFAULT_ITERATIONS

        # Step 4: Derive key and encrypt
        key = cls.derive_key(password, salt, iterations)
        aesgcm = AESGCM(key)
        ciphertext = aesgcm.encrypt(nonce, payload_bytes, None)

        # Step 5: Create comprehensive package
        package = {
            'version': cls.VERSION,
            'algorithm': cls.ALGORITHM,
            'timestamp': int(time.time()),
            'salt': base64.b64encode(salt).decode('ascii'),
            'nonce': base64.b64encode(nonce).decode('ascii'),
            'ciphertext': base64.b64encode(ciphertext).decode('ascii'),
            'iterations': iterations,
            'compressed': compressed,
            'compression_level': compression_level,
            'compression_info': compression_info,
            'original_size': len(plaintext),
            'payload_size': len(payload_bytes),
            'encrypted_size': len(ciphertext),
            'message_hash': message_hash,
            'auto_compress': auto_compress
        }

        return json.dumps(package, separators=(',', ':')).encode('utf-8')

    @classmethod
    def decrypt(cls, encrypted_data: bytes, password: str, verify_integrity: bool = True) -> bytes:
        """
        Decrypt data with automatic decompression and integrity verification
        """
        if not isinstance(encrypted_data, bytes):
            raise ValueError("Encrypted data must be bytes")

        try:
            # Parse JSON package
            json_str = encrypted_data.decode('utf-8')
            package = json.loads(json_str)
        except (json.JSONDecodeError, UnicodeDecodeError) as e:
            raise ValueError(f"Invalid package format: {e}")

        # Validate package
        if package.get('algorithm') not in [cls.ALGORITHM, "AES-256-GCM"]:
            raise ValueError(f"Unsupported algorithm: {package.get('algorithm')}")

        # Extract components
        try:
            salt = base64.b64decode(package['salt'])
            nonce = base64.b64decode(package['nonce'])
            ciphertext = base64.b64decode(package['ciphertext'])
        except (KeyError, ValueError) as e:
            raise ValueError(f"Invalid package components: {e}")

        iterations = package.get('iterations', cls.DEFAULT_ITERATIONS)
        compressed = package.get('compressed', False)
        message_hash = package.get('message_hash', '')
        compression_level = package.get('compression_level', 0)

        # Derive key and decrypt
        key = cls.derive_key(password, salt, iterations)
        aesgcm = AESGCM(key)

        try:
            decrypted_payload = aesgcm.decrypt(nonce, ciphertext, None)
        except Exception:
            raise ValueError("Decryption failed - invalid password or corrupted data")

        # Handle decompression
        plaintext = decrypted_payload
        if compressed:
            try:
                plaintext = zlib.decompress(decrypted_payload)
                print(f"🗜️ Auto-decompressed from level {compression_level}")
            except zlib.error as e:
                raise ValueError(f"Decompression failed: {e}")

        # Verify integrity
        if verify_integrity and message_hash:
            calculated_hash = hashlib.sha256(plaintext).hexdigest()
            if calculated_hash != message_hash:
                raise ValueError("SHA-256 integrity verification failed")
            print(f"✅ Integrity verified (SHA-256: {message_hash[:16]}...)")

        return plaintext

    @classmethod
    def estimate_size(cls, plaintext_size: int, auto_compress: bool = True) -> Dict[str, int]:
        """
        Estimate encrypted package size with compression analysis
        """
        if plaintext_size <= 0:
            return {'encrypted_size': 0, 'compression_used': False}

        # Estimate compression (conservative)
        effective_size = plaintext_size
        compression_used = False

        if auto_compress and plaintext_size > 100:
            # Conservative compression estimate
            estimated_ratio = 0.75 if plaintext_size > 1000 else 0.85
            compressed_size = int(plaintext_size * estimated_ratio)

            if compressed_size < plaintext_size * 0.95:  # 5% minimum savings
                effective_size = compressed_size
                compression_used = True

        # Calculate encryption overhead
        ciphertext_size = effective_size + 16  # GCM auth tag

        # Base64 encoding overhead
        salt_b64 = 24
        nonce_b64 = 16
        ciphertext_b64 = ((ciphertext_size + 2) // 3) * 4

        # JSON overhead (metadata)
        json_overhead = 500

        total_size = json_overhead + salt_b64 + nonce_b64 + ciphertext_b64

        return {
            'encrypted_size': total_size,
            'original_size': plaintext_size,
            'effective_size': effective_size,
            'compression_used': compression_used,
            'compression_savings': plaintext_size - effective_size if compression_used else 0
        }

# ============================================================================
# ENHANCED PVD STEGANOGRAPHY ENGINE
# ============================================================================

# PVD Protocol Constants
PVD_MAGIC = [0x53, 0x50, 0x56, 0x44]  # "SPVD" (Smart PVD)
PVD_VERSION = [3, 0, 0]
PVD_HEADER_SIZE = 16  # 4 (magic) + 3 (version) + 8 (length) + 1 (checksum)

class EnhancedBitStream:
    """Enhanced bit stream handling with validation"""

    def __init__(self, data_bytes: bytes = None):
        if data_bytes is not None:
            self._setup_reader(data_bytes)
        else:
            self._setup_writer()

    def _setup_reader(self, data_bytes: bytes):
        """Setup for reading mode"""
        if not isinstance(data_bytes, bytes) or len(data_bytes) == 0:
            raise ValueError("Data must be non-empty bytes")

        data_len = len(data_bytes)
        if data_len > 0xFFFFFFFFFFFFFFFF:
            raise ValueError("Data too large")

        # Calculate simple checksum
        checksum = sum(data_bytes) & 0xFF

        header = bytes(PVD_MAGIC + PVD_VERSION +
                      list(data_len.to_bytes(8, 'big')) + [checksum])
        full_data = header + data_bytes

        # Convert to bit stream
        self.bit_stream = ''.join(f'{byte:08b}' for byte in full_data)
        self.position = 0
        self.total_bits = len(self.bit_stream)
        self.data_size = len(data_bytes)
        self._mode = 'read'

    def _setup_writer(self):
        """Setup for writing mode"""
        self.bits_buffer = ''
        self.bytes_list = []
        self.header_processed = False
        self.expected_length = 0
        self.expected_checksum = 0
        self.valid_header = False
        self._mode = 'write'

    def read_bits(self, count: int) -> Tuple[bool, int, int]:
        """Read bits from stream (reader mode)"""
        if self._mode != 'read':
            raise ValueError("Stream not in read mode")

        if not 1 <= count <= 8:
            raise ValueError("Bit count must be 1-8")

        if self.position >= self.total_bits:
            return True, 0, 0  # EOF

        available = self.total_bits - self.position
        to_read = min(count, available)

        bit_slice = self.bit_stream[self.position:self.position + to_read]
        value = int(bit_slice, 2) if bit_slice else 0

        self.position += to_read
        return (self.position >= self.total_bits), value, to_read

    def write_bits(self, count: int, value: int):
        """Write bits to stream (writer mode)"""
        if self._mode != 'write':
            raise ValueError("Stream not in write mode")

        if not 1 <= count <= 8 or value < 0 or value >= (1 << count):
            raise ValueError("Invalid bit count or value")

        # Add to buffer
        self.bits_buffer += f'{value:0{count}b}'

        # Process complete bytes
        while len(self.bits_buffer) >= 8:
            byte_bits = self.bits_buffer[:8]
            self.bytes_list.append(int(byte_bits, 2))
            self.bits_buffer = self.bits_buffer[8:]

            # Process header when available
            if not self.header_processed and len(self.bytes_list) >= PVD_HEADER_SIZE:
                self._process_header()

    def _process_header(self):
        """Process and validate header"""
        if len(self.bytes_list) < PVD_HEADER_SIZE:
            return

        try:
            magic = self.bytes_list[:4]
            version = self.bytes_list[4:7]
            length_bytes = self.bytes_list[7:15]
            checksum = self.bytes_list[15]

            if magic != PVD_MAGIC:
                raise ValueError(f"Invalid magic: {magic}")

            if version not in [PVD_VERSION, [2, 1, 0], [2, 0, 0]]:
                print(f"⚠️ Version mismatch: {version}")

            self.expected_length = int.from_bytes(bytes(length_bytes), 'big')
            self.expected_checksum = checksum
            self.header_processed = True
            self.valid_header = True

            print(f"✅ Header validated - expecting {self.expected_length} bytes")

        except Exception as e:
            raise ValueError(f"Header validation failed: {e}")

    def is_complete(self) -> bool:
        """Check if extraction is complete"""
        if not self.header_processed:
            return False

        total_expected = PVD_HEADER_SIZE + self.expected_length
        return len(self.bytes_list) >= total_expected

    def get_data(self) -> bytes:
        """Get extracted data"""
        if not self.is_complete():
            raise ValueError("Stream not complete")

        # Extract payload
        payload = bytes(self.bytes_list[PVD_HEADER_SIZE:PVD_HEADER_SIZE + self.expected_length])

        # Verify checksum
        calculated_checksum = sum(payload) & 0xFF
        if calculated_checksum != self.expected_checksum:
            print(f"⚠️ Checksum mismatch: expected {self.expected_checksum}, got {calculated_checksum}")

        return payload

    def progress(self) -> float:
        """Get progress percentage"""
        if self._mode == 'read':
            return (self.position / self.total_bits) * 100 if self.total_bits > 0 else 0
        else:
            if not self.header_processed or self.expected_length == 0:
                return 0
            current_data = max(0, len(self.bytes_list) - PVD_HEADER_SIZE)
            return min(100, (current_data / self.expected_length) * 100)

class AdvancedPVD:
    """Advanced Pixel Value Differencing steganography engine"""

    @staticmethod
    def get_capacity(diff: int) -> int:
        """Calculate bit capacity based on pixel difference"""
        abs_diff = abs(diff)
        if abs_diff < 4:
            return 1
        elif abs_diff < 8:
            return 2
        elif abs_diff < 16:
            return 3
        elif abs_diff < 32:
            return 4
        else:
            return 5

    @staticmethod
    def calculate_capacity(image: Image.Image) -> Dict[str, Any]:
        """Calculate detailed embedding capacity"""
        if image.mode != 'RGB':
            image = image.convert('RGB')

        width, height = image.size
        if width < 6 or height < 6:
            return {
                'total_bits': 0,
                'usable_bytes': 0,
                'blocks': 0,
                'efficiency': 0,
                'error': 'Image too small (minimum 6x6 pixels required)'
            }

        pixels = image.load()

        # Process in 3x3 blocks
        blocks_x = (width // 3) - 1
        blocks_y = (height // 3) - 1

        if blocks_x < 1 or blocks_y < 1:
            return {
                'total_bits': 0,
                'usable_bytes': 0,
                'blocks': 0,
                'efficiency': 0,
                'error': 'Not enough blocks for embedding'
            }

        total_bits = 0
        capacity_dist = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}

        for by in range(blocks_y):
            for bx in range(blocks_x):
                # Get block center as reference
                ref_x, ref_y = bx * 3 + 1, by * 3 + 1
                ref_pixel = pixels[ref_x, ref_y]

                # Process surrounding pixels
                for dy in range(3):
                    for dx in range(3):
                        if dx == 1 and dy == 1:
                            continue

                        curr_pixel = pixels[bx * 3 + dx, by * 3 + dy]

                        # Calculate capacity for each RGB channel
                        for channel in range(3):
                            diff = curr_pixel[channel] - ref_pixel[channel]
                            capacity = AdvancedPVD.get_capacity(diff)
                            total_bits += capacity
                            capacity_dist[capacity] += 1

        # Account for header
        header_bits = PVD_HEADER_SIZE * 8
        usable_bits = max(0, total_bits - header_bits)
        usable_bytes = usable_bits // 8

        return {
            'total_bits': total_bits,
            'header_bits': header_bits,
            'usable_bits': usable_bits,
            'usable_bytes': usable_bytes,
            'blocks': blocks_x * blocks_y,
            'efficiency': (usable_bits / total_bits * 100) if total_bits > 0 else 0,
            'capacity_distribution': capacity_dist
        }

    @staticmethod
    def embed_bits(pixel_value: int, bit_count: int, data: int) -> int:
        """Embed data bits into pixel value"""
        if bit_count == 0:
            return pixel_value

        mask = (1 << bit_count) - 1
        new_value = (pixel_value & ~mask) | (data & mask)
        return max(0, min(255, new_value))

    @staticmethod
    def extract_bits(pixel_value: int, bit_count: int) -> int:
        """Extract data bits from pixel value"""
        if bit_count == 0:
            return 0
        mask = (1 << bit_count) - 1
        return pixel_value & mask

    def embed(self, cover: Image.Image, data: bytes,
              progress_callback: callable = None) -> Image.Image:
        """Embed encrypted data using advanced PVD"""
        if cover.mode != 'RGB':
            cover = cover.convert('RGB')

        # Validate capacity
        capacity = self.calculate_capacity(cover)
        if 'error' in capacity:
            raise ValueError(f"Image unsuitable: {capacity['error']}")

        if len(data) > capacity['usable_bytes']:
            raise ValueError(f"Data too large: {len(data)} > {capacity['usable_bytes']}")

        # Setup bit stream
        bit_reader = EnhancedBitStream(data)

        # Create output image
        stego = cover.copy()
        cover_pixels = cover.load()
        stego_pixels = stego.load()

        width, height = cover.size
        blocks_x = (width // 3) - 1
        blocks_y = (height // 3) - 1

        total_blocks = blocks_x * blocks_y
        processed_blocks = 0
        embedding_done = False

        # Process blocks
        for by in range(blocks_y):
            if embedding_done:
                break

            for bx in range(blocks_x):
                if embedding_done:
                    break

                # Get reference pixel
                ref_x, ref_y = bx * 3 + 1, by * 3 + 1
                ref_pixel = cover_pixels[ref_x, ref_y]

                # Process surrounding pixels
                for dy in range(3):
                    if embedding_done:
                        break

                    for dx in range(3):
                        if dx == 1 and dy == 1 or embedding_done:
                            continue

                        px, py = bx * 3 + dx, by * 3 + dy
                        current_pixel = list(cover_pixels[px, py])

                        # Process each color channel
                        for ch in range(3):
                            eof, value, bits_read = bit_reader.read_bits(
                                self.get_capacity(current_pixel[ch] - ref_pixel[ch])
                            )

                            if bits_read > 0:
                                current_pixel[ch] = self.embed_bits(
                                    current_pixel[ch], bits_read, value
                                )

                            if eof:
                                embedding_done = True
                                break

                        stego_pixels[px, py] = tuple(current_pixel)

                processed_blocks += 1

                # Progress callback
                if progress_callback and processed_blocks % 50 == 0:
                    block_progress = (processed_blocks / total_blocks) * 100
                    bit_progress = bit_reader.progress()
                    progress_callback(block_progress, bit_progress)

        return stego

    def extract(self, cover: Image.Image, stego: Image.Image,
                progress_callback: callable = None) -> bytes:
        """Extract encrypted data using advanced PVD"""
        if cover.size != stego.size:
            raise ValueError("Image dimensions must match")

        if cover.mode != 'RGB':
            cover = cover.convert('RGB')
        if stego.mode != 'RGB':
            stego = stego.convert('RGB')

        cover_pixels = cover.load()
        stego_pixels = stego.load()

        width, height = cover.size
        blocks_x = (width // 3) - 1
        blocks_y = (height // 3) - 1

        # Setup bit stream writer
        bit_writer = EnhancedBitStream()

        total_blocks = blocks_x * blocks_y
        processed_blocks = 0

        # Process blocks
        for by in range(blocks_y):
            for bx in range(blocks_x):
                # Get reference pixel
                ref_x, ref_y = bx * 3 + 1, by * 3 + 1
                ref_pixel = cover_pixels[ref_x, ref_y]

                # Process surrounding pixels
                for dy in range(3):
                    for dx in range(3):
                        if dx == 1 and dy == 1:
                            continue

                        px, py = bx * 3 + dx, by * 3 + dy
                        cover_pixel = cover_pixels[px, py]
                        stego_pixel = stego_pixels[px, py]

                        # Extract from each channel
                        for ch in range(3):
                            diff = cover_pixel[ch] - ref_pixel[ch]
                            capacity = self.get_capacity(diff)

                            if capacity > 0:
                                extracted = self.extract_bits(stego_pixel[ch], capacity)
                                bit_writer.write_bits(capacity, extracted)

                                if bit_writer.is_complete():
                                    return bit_writer.get_data()

                processed_blocks += 1

                # Progress callback
                if progress_callback and processed_blocks % 50 == 0:
                    block_progress = (processed_blocks / total_blocks) * 100
                    extraction_progress = bit_writer.progress()
                    progress_callback(block_progress, extraction_progress)

        return b''  # No complete data found

# ============================================================================
# REAL-TIME CAPACITY MONITORING SYSTEM
# ============================================================================

class IntelligentCapacityMonitor:
    """Real-time capacity monitoring with intelligent warnings"""

    WARNING_LEVELS = {
        'safe': (0, 70, '#28a745'),      # Green
        'moderate': (70, 85, '#17a2b8'),  # Blue
        'high': (85, 95, '#ffc107'),      # Yellow
        'critical': (95, 98, '#fd7e14'),  # Orange
        'danger': (98, 100, '#dc3545')    # Red
    }

    def __init__(self, capacity_bytes: int):
        self.capacity_bytes = max(1, capacity_bytes)

    def analyze(self, message: str, auto_compress: bool = True) -> Dict[str, Any]:
        """Analyze message capacity with real-time feedback"""
        if not message.strip():
            return {
                'status': 'empty',
                'usage_percent': 0,
                'level': 'safe',
                'can_fit': True,
                'message': 'Start typing your message...'
            }

        # Calculate sizes
        try:
            message_bytes = message.encode('utf-8')
            size_info = MilitaryAESGCM.estimate_size(len(message_bytes), auto_compress)

            usage_percent = (size_info['encrypted_size'] / self.capacity_bytes) * 100
            can_fit = size_info['encrypted_size'] <= self.capacity_bytes

            # Determine warning level
            level = 'safe'
            for level_name, (min_pct, max_pct, color) in self.WARNING_LEVELS.items():
                if min_pct <= usage_percent < max_pct:
                    level = level_name
                    break

            # Generate status message
            if not can_fit:
                status_msg = f"❌ OVERFLOW: Need {size_info['encrypted_size']:,} bytes, have {self.capacity_bytes:,}"
            elif level == 'danger':
                status_msg = f"🚨 CRITICAL: {usage_percent:.1f}% - Immediate action required!"
            elif level == 'critical':
                status_msg = f"⚠️ HIGH RISK: {usage_percent:.1f}% - Consider shortening message"
            elif level == 'high':
                status_msg = f"⚠️ MODERATE: {usage_percent:.1f}% - Approaching capacity limit"
            elif level == 'moderate':
                status_msg = f"📊 GOOD: {usage_percent:.1f}% - Reasonable usage"
            else:
                status_msg = f"✅ EXCELLENT: {usage_percent:.1f}% - Plenty of space"

            return {
                'status': 'ready' if can_fit else 'overflow',
                'usage_percent': usage_percent,
                'estimated_size': size_info['encrypted_size'],
                'original_size': len(message_bytes),
                'compression_used': size_info['compression_used'],
                'compression_savings': size_info['compression_savings'],
                'can_fit': can_fit,
                'level': level,
                'color': self.WARNING_LEVELS[level][2],
                'message': status_msg,
                'bytes_remaining': max(0, self.capacity_bytes - size_info['encrypted_size'])
            }
        except Exception as e:
            return {
                'status': 'error',
                'usage_percent': 0,
                'level': 'danger',
                'can_fit': False,
                'message': f'Error analyzing message: {str(e)}'
            }

# ============================================================================
# ENHANCED GUI SYSTEM
# ============================================================================

class AdvancedBengaliSteganographyGUI:
    """Advanced GUI with real-time monitoring and colorful buttons"""

    def __init__(self):
        self.pvd = AdvancedPVD()
        self.capacity_monitor = None
        self.encoded_image = None
        self.setup_gui()

    def setup_gui(self):
        """Initialize the enhanced GUI interface"""
        self._apply_styles()
        self._create_tabs()

    def _apply_styles(self):
        """Apply enhanced CSS styles with colorful buttons"""
        display(HTML("""
        <style>
        .main-title {
            font-size: 36px;
            font-weight: 900;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            text-align: center;
            margin: 30px 0 10px 0;
            text-shadow: 3px 3px 6px rgba(0,0,0,0.2);
            letter-spacing: 2px;
        }
        .subtitle {
            text-align: center;
            font-size: 18px;
            font-weight: 600;
            color: #2c3e50;
            margin-bottom: 30px;
            background: linear-gradient(90deg, #ff6b6b, #4ecdc4, #45b7d1);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            letter-spacing: 1px;
        }
        .smart-section {
            background: linear-gradient(135deg, #f8f9ff, #f0f4ff);
            padding: 25px;
            border-radius: 20px;
            margin: 20px 0;
            border: 2px solid #e6f0ff;
            box-shadow: 0 8px 25px rgba(0,0,0,0.1);
            transition: transform 0.3s ease, box-shadow 0.3s ease;
        }
        .smart-section:hover {
            transform: translateY(-2px);
            box-shadow: 0 12px 35px rgba(0,0,0,0.15);
        }
        .capacity-monitor {
            background: white;
            padding: 25px;
            border-radius: 15px;
            margin: 20px 0;
            border-left: 5px solid #2575fc;
            box-shadow: 0 5px 15px rgba(0,0,0,0.1);
        }
        .capacity-bar {
            width: 100%;
            height: 35px;
            background: linear-gradient(135deg, #f8f9fa, #e9ecef);
            border-radius: 20px;
            overflow: hidden;
            margin: 15px 0;
            border: 2px solid #dee2e6;
        }
        .capacity-fill {
            height: 100%;
            transition: all 0.5s cubic-bezier(0.4, 0, 0.2, 1);
            display: flex;
            align-items: center;
            justify-content: center;
            color: white;
            font-weight: bold;
            font-size: 16px;
            text-shadow: 1px 1px 2px rgba(0,0,0,0.3);
        }
        .level-safe { background: linear-gradient(90deg, #28a745, #20c997); }
        .level-moderate { background: linear-gradient(90deg, #17a2b8, #138496); }
        .level-high { background: linear-gradient(90deg, #ffc107, #e0a800); }
        .level-critical { background: linear-gradient(90deg, #fd7e14, #e55100); }
        .level-danger {
            background: linear-gradient(90deg, #dc3545, #c82333);
            animation: pulse 1.5s infinite;
        }
        .smart-card {
            background: white;
            padding: 20px;
            border-radius: 12px;
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
            margin: 10px 0;
            border-left: 4px solid #6a11cb;
        }
        .feature-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 20px;
            margin: 25px 0;
        }
        .feature-card {
            background: white;
            padding: 20px;
            border-radius: 15px;
            box-shadow: 0 3px 10px rgba(0,0,0,0.1);
            transition: transform 0.3s ease;
        }
        .feature-card:hover {
            transform: translateY(-5px);
            box-shadow: 0 8px 25px rgba(0,0,0,0.15);
        }
        .step-indicator {
            display: inline-block;
            width: 35px;
            height: 35px;
            border-radius: 50%;
            background: linear-gradient(135deg, #667eea, #764ba2);
            color: white;
            text-align: center;
            line-height: 35px;
            font-weight: bold;
            margin-right: 15px;
        }
        .success-animation {
            animation: bounceIn 1s ease-out;
        }

        /* Colorful Button Styles */
        .btn-encode {
            background: linear-gradient(45deg, #FF6B6B, #4ECDC4) !important;
            border: none !important;
            color: white !important;
            font-weight: bold !important;
            font-size: 16px !important;
            text-shadow: 1px 1px 2px rgba(0,0,0,0.3) !important;
            box-shadow: 0 8px 25px rgba(255,107,107,0.4) !important;
            transition: all 0.3s ease !important;
        }
        .btn-encode:hover {
            transform: translateY(-3px) !important;
            box-shadow: 0 12px 35px rgba(255,107,107,0.6) !important;
        }
        .btn-decode {
            background: linear-gradient(45deg, #667eea, #764ba2) !important;
            border: none !important;
            color: white !important;
            font-weight: bold !important;
            font-size: 16px !important;
            text-shadow: 1px 1px 2px rgba(0,0,0,0.3) !important;
            box-shadow: 0 8px 25px rgba(102,126,234,0.4) !important;
            transition: all 0.3s ease !important;
        }
        .btn-decode:hover {
            transform: translateY(-3px) !important;
            box-shadow: 0 12px 35px rgba(102,126,234,0.6) !important;
        }
        .btn-download {
            background: linear-gradient(45deg, #11998e, #38ef7d) !important;
            border: none !important;
            color: white !important;
            font-weight: bold !important;
            font-size: 16px !important;
            text-shadow: 1px 1px 2px rgba(0,0,0,0.3) !important;
            box-shadow: 0 8px 25px rgba(17,153,142,0.4) !important;
            transition: all 0.3s ease !important;
        }
        .btn-download:hover {
            transform: translateY(-3px) !important;
            box-shadow: 0 12px 35px rgba(17,153,142,0.6) !important;
        }
        .btn-disabled {
            background: linear-gradient(45deg, #95a5a6, #bdc3c7) !important;
            border: none !important;
            color: white !important;
            opacity: 0.6 !important;
            cursor: not-allowed !important;
        }

        @keyframes pulse {
            0%, 100% { opacity: 1; }
            50% { opacity: 0.7; }
        }
        @keyframes bounceIn {
            0% { transform: scale(0.3); opacity: 0; }
            50% { transform: scale(1.05); }
            70% { transform: scale(0.9); }
            100% { transform: scale(1); opacity: 1; }
        }

        /* Upload widget styling */
        .widget-upload {
            border: 3px dashed #667eea !important;
            border-radius: 15px !important;
            background: linear-gradient(135deg, #f8f9ff, #f0f4ff) !important;
            transition: all 0.3s ease !important;
        }
        .widget-upload:hover {
            border-color: #4ECDC4 !important;
            background: linear-gradient(135deg, #e8f5e9, #d4edda) !important;
        }

        /* Text area styling */
        .widget-textarea textarea {
            border: 2px solid #e9ecef !important;
            border-radius: 10px !important;
            transition: border-color 0.3s ease !important;
        }
        .widget-textarea textarea:focus {
            border-color: #667eea !important;
            box-shadow: 0 0 10px rgba(102,126,234,0.2) !important;
        }
        </style>

        <div class="main-title">🛡️ Advanced Bengali Steganography Suite 🛡️</div>
        <div class="subtitle">Military Grade AES-256-GCM + Compression</div>
        """))

    def _create_tabs(self):
        """Create tabbed interface"""
        encode_tab = self._create_encode_tab()
        decode_tab = self._create_decode_tab()

        tabs = widgets.Tab()
        tabs.children = [encode_tab, decode_tab]
        tabs.titles = ['🔐 Encode Secret Message', '🔓 Decode Hidden Message']

        display(tabs)

    def _create_encode_tab(self):
        """Create encoding interface with colorful buttons"""
        # Step 1: Image Upload
        self.upload_widget = widgets.FileUpload(
            accept='image/*',
            multiple=False,
            description='📤 Select Cover Image',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='100%', height='60px')
        )
        self.image_analysis = widgets.HTML(value='')

        # Step 2: Password Setup
        self.password = widgets.Password(
            placeholder='Enter strong encryption password (minimum 8 characters)',
            description='🔐 Password:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='100%'),
            disabled=True
        )
        self.password_confirm = widgets.Password(
            placeholder='Confirm your password',
            description='🔒 Confirm:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='100%'),
            disabled=True
        )
        self.password_status = widgets.HTML(value='')

        # Step 3: Message Input with Real-time Monitoring
        self.message = widgets.Textarea(
            value='',
            placeholder='আপনার গোপন বার্তা এখানে লিখুন...\n\nType your secret message here in Bengali or English.\nReal-time capacity monitoring will show below! 🚀',
            description='📝 Secret Message:',
            style={'description_width': '140px'},
            layout=widgets.Layout(width='100%', height='300px'),
            disabled=True
        )

        self.capacity_display = widgets.HTML(value='')
        self.message_analysis = widgets.HTML(value='')

        # Step 4: Smart Options
        self.auto_compress = widgets.Checkbox(
            value=True,
            description='🤖 Enable intelligent auto-compression',
            style={'description_width': 'initial'},
            disabled=True
        )
        self.sha256_verify = widgets.Checkbox(
            value=True,
            description='🔍 Enable SHA-256 message integrity verification',
            style={'description_width': 'initial'},
            disabled=True
        )

        # Step 5: Processing - Colorful Button
        self.encode_button = widgets.Button(
            description='🔐 Military Encrypt & Embed',
            disabled=True,
            layout=widgets.Layout(width='350px', height='60px'),
            icon='shield-alt'
        )
        self.encode_progress = widgets.HTML(value='')

        # Step 6: Download - Colorful Button
        self.download_button = widgets.Button(
            description='💾 Download Secure Image',
            disabled=True,
            layout=widgets.Layout(width='350px', height='60px'),
            icon='download'
        )

        # Output
        self.encode_output = widgets.Output()

        # Event handlers
        self.upload_widget.observe(self._on_image_upload, names='value')
        self.password.observe(self._on_password_change, names='value')
        self.password_confirm.observe(self._on_password_change, names='value')
        self.message.observe(self._on_message_change, names='value')
        self.auto_compress.observe(self._on_options_change, names='value')
        self.sha256_verify.observe(self._on_options_change, names='value')
        self.encode_button.on_click(self._perform_encoding)
        self.download_button.on_click(self._download_image)

        # Layout
        encode_tab_content = widgets.VBox([
            widgets.HTML('<div class="smart-section"><h3><span class="step-indicator">1</span>Upload Cover Image</h3></div>'),
            self.upload_widget,
            self.image_analysis,

            widgets.HTML('<div class="smart-section"><h3><span class="step-indicator">2</span>Configure Military Encryption</h3></div>'),
            self.password,
            self.password_confirm,
            self.password_status,

            widgets.HTML('<div class="smart-section"><h3><span class="step-indicator">3</span>Enter Secret Message</h3></div>'),
            self.message,
            self.capacity_display,
            self.message_analysis,

            widgets.HTML('<div class="smart-section"><h3><span class="step-indicator">4</span>Advanced Options</h3></div>'),
            self.auto_compress,
            self.sha256_verify,

            widgets.HTML('<div class="smart-section"><h3><span class="step-indicator">5</span>Process & Generate</h3></div>'),
            widgets.HBox([self.encode_button], layout=widgets.Layout(justify_content='center')),
            self.encode_progress,

            widgets.HTML('<div class="smart-section"><h3><span class="step-indicator">6</span>Download Result</h3></div>'),
            widgets.HBox([self.download_button], layout=widgets.Layout(justify_content='center')),

            widgets.HTML('<div class="smart-section"><h3>📋 Processing Output</h3></div>'),
            self.encode_output
        ])

        return encode_tab_content

    def _create_decode_tab(self):
        """Create decoding interface with colorful buttons"""
        # File uploads
        self.cover_upload = widgets.FileUpload(
            accept='image/*',
            multiple=False,
            description='📤 Original Cover Image',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='100%', height='60px')
        )

        self.stego_upload = widgets.FileUpload(
            accept='image/*',
            multiple=False,
            description='🔒 Encoded Image',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='100%', height='60px')
        )

        self.decode_password = widgets.Password(
            placeholder='Enter the decryption password',
            description='🔐 Password:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='100%')
        )

        # Options
        self.verify_integrity = widgets.Checkbox(
            value=True,
            description='🔍 Verify SHA-256 message integrity'
        )
        self.detailed_analysis = widgets.Checkbox(
            value=True,
            description='📊 Show detailed analysis'
        )

        # Processing - Colorful Button
        self.decode_button = widgets.Button(
            description='🔓 Military Extract & Decrypt',
            disabled=True,
            layout=widgets.Layout(width='350px', height='60px'),
            icon='unlock-alt'
        )

        # Output
        self.decode_progress = widgets.HTML(value='')
        self.decode_info = widgets.HTML(value='')
        self.decoded_message = widgets.HTML(value='')
        self.decode_output = widgets.Output()

        # Event handlers
        self.cover_upload.observe(self._on_decode_uploads, names='value')
        self.stego_upload.observe(self._on_decode_uploads, names='value')
        self.decode_password.observe(self._on_decode_password, names='value')
        self.decode_button.on_click(self._perform_decoding)

        # Apply button styles after all buttons are created
        self._apply_button_styles()

        # Layout
        decode_tab_content = widgets.VBox([
            widgets.HTML('<div class="smart-section"><h3>🔍 Advanced Decoding Process</h3><p>Upload both images and enter the password to extract and decrypt your hidden message with automatic verification.</p></div>'),

            widgets.HTML('<div class="smart-section"><h3><span class="step-indicator">1</span>Upload Images</h3></div>'),
            widgets.HBox([self.cover_upload, self.stego_upload]),
            self.decode_info,

            widgets.HTML('<div class="smart-section"><h3><span class="step-indicator">2</span>Decryption Settings</h3></div>'),
            self.decode_password,
            widgets.HTML('<h4>🔧 Advanced Options:</h4>'),
            self.verify_integrity,
            self.detailed_analysis,

            widgets.HTML('<div class="smart-section"><h3><span class="step-indicator">3</span>Extract & Decrypt</h3></div>'),
            widgets.HBox([self.decode_button], layout=widgets.Layout(justify_content='center')),
            self.decode_progress,

            widgets.HTML('<div class="smart-section"><h3>🔓 Decrypted Message</h3></div>'),
            self.decoded_message,
            self.decode_output
        ])

        return decode_tab_content

    def _apply_button_styles(self):
        """Apply colorful styles to buttons after they're all created"""
        # Apply encode button style
        if hasattr(self, 'encode_button'):
            self.encode_button.add_class('btn-encode')

        # Apply download button style
        if hasattr(self, 'download_button'):
            self.download_button.add_class('btn-download')

        # Apply decode button style
        if hasattr(self, 'decode_button'):
            self.decode_button.add_class('btn-decode')

    # Event Handlers
    def _on_image_upload(self, change):
        """Handle image upload and analysis"""
        if not CRYPTO_AVAILABLE:
            self.image_analysis.value = '''
            <div class="smart-card" style="border-left-color: #dc3545;">
                <strong>❌ Cryptography Library Required</strong><br>
                Please install: <code>pip install cryptography</code>
            </div>
            '''
            return

        if change['new']:
            try:
                uploaded_file = list(change['new'].values())[0]
                self.cover_image = Image.open(io.BytesIO(uploaded_file['content'])).convert('RGB')

                # Analyze capacity
                capacity_info = self.pvd.calculate_capacity(self.cover_image)

                if 'error' in capacity_info:
                    self.image_analysis.value = f'''
                    <div class="smart-card" style="border-left-color: #dc3545;">
                        <strong>❌ Image Analysis Failed</strong><br>
                        Error: {capacity_info['error']}
                    </div>
                    '''
                    return

                self.capacity_monitor = IntelligentCapacityMonitor(capacity_info['usable_bytes'])

                # Display analysis
                file_size = len(uploaded_file['content'])
                analysis_html = f'''
                <div class="capacity-monitor success-animation">
                    <h4>📊 Military Grade Image Analysis Complete</h4>
                    <div class="feature-grid">
                        <div class="feature-card">
                            <strong>📏 Dimensions</strong><br>
                            {self.cover_image.size[0]} × {self.cover_image.size[1]} pixels
                        </div>
                        <div class="feature-card">
                            <strong>💾 File Size</strong><br>
                            {file_size:,} bytes ({file_size/1024:.1f} KB)
                        </div>
                        <div class="feature-card">
                            <strong>🎯 Military Capacity</strong><br>
                            {capacity_info['usable_bytes']:,} bytes
                        </div>
                        <div class="feature-card">
                            <strong>⚡ Efficiency</strong><br>
                            {capacity_info['efficiency']:.1f}%
                        </div>
                    </div>
                    <div style="background: #d4edda; padding: 15px; border-radius: 10px; margin-top: 15px;">
                        <strong>✅ Image ready for military-grade steganography!</strong><br>
                        Estimated capacity: ~{capacity_info['usable_bytes'] // 3:,} Bengali characters with compression
                    </div>
                </div>
                '''

                self.image_analysis.value = analysis_html

                # Enable next step
                self.password.disabled = False
                self.password_confirm.disabled = False

                with self.encode_output:
                    clear_output()
                    print("✅ Step 1 Complete: Military image analysis finished!")
                    print(f"🎯 Capacity: {capacity_info['usable_bytes']:,} bytes")
                    print("🚀 Proceed to Step 2: Configure encryption")

            except Exception as e:
                self.image_analysis.value = f'''
                <div class="smart-card" style="border-left-color: #dc3545;">
                    <strong>❌ Image Analysis Failed</strong><br>
                    Error: {str(e)}
                </div>
                '''

    def _on_password_change(self, change):
        """Handle password validation"""
        password = self.password.value
        confirm = self.password_confirm.value

        if not password:
            self.password_status.value = ''
            self._update_encoding_state()
            return

        # Analyze password strength
        strength = self._analyze_password(password)
        match_valid = password == confirm if confirm else None

        # Update status
        if strength['score'] >= 3 and match_valid and len(password) >= 8:
            color = "#28a745"
            status = "🛡️ Military Grade Password - Security Ready!"
            self.message.disabled = False
            self.auto_compress.disabled = False
            self.sha256_verify.disabled = False
        elif match_valid is False:
            color = "#dc3545"
            status = "❌ Passwords don't match"
            self.message.disabled = True
        else:
            color = "#ffc107"
            status = "⚠️ Password needs improvement"
            self.message.disabled = True

        status_html = f'''
        <div class="smart-card" style="border-left-color: {color};">
            <h4 style="color: {color};">{status}</h4>
            <div style="margin: 15px 0;">
                <strong>Password Strength: {strength['score']}/5</strong><br>
                {"<br>".join(strength['factors'])}
            </div>
        '''

        if match_valid is not None:
            match_icon = "✅" if match_valid else "❌"
            status_html += f'<div><strong>Password Match:</strong> {match_icon}</div>'

        status_html += '</div>'
        self.password_status.value = status_html

        self._update_encoding_state()

    def _analyze_password(self, password: str) -> Dict[str, Any]:
        """Analyze password strength"""
        score = 0
        factors = []

        if len(password) >= 8:
            score += 1
            factors.append("✅ Adequate length (8+ chars)")
        else:
            factors.append("❌ Too short (need 8+ chars)")

        if any(c.islower() for c in password):
            score += 1
            factors.append("✅ Lowercase letters")

        if any(c.isupper() for c in password):
            score += 1
            factors.append("✅ Uppercase letters")

        if any(c.isdigit() for c in password):
            score += 1
            factors.append("✅ Numbers")

        if any(c in "!@#$%^&*()_+-=[]{}|;:,.<>?" for c in password):
            score += 1
            factors.append("✅ Special characters")

        return {'score': score, 'factors': factors}

    def _on_message_change(self, change):
        """Handle real-time message analysis"""
        message = change['new']

        if not self.capacity_monitor:
            return

        # Analyze capacity
        auto_compress = self.auto_compress.value if hasattr(self, 'auto_compress') else True
        analysis = self.capacity_monitor.analyze(message, auto_compress)

        # Update capacity display
        self._update_capacity_display(analysis)

        # Update message analysis
        self._update_message_analysis(message, analysis)

        # Update encoding button state
        self._update_encoding_state()

    def _update_capacity_display(self, analysis: Dict[str, Any]):
        """Update real-time capacity display"""
        if analysis['status'] == 'empty':
            self.capacity_display.value = '''
            <div class="capacity-monitor">
                <h4>📊 Real-time Military Capacity Monitor</h4>
                <div class="capacity-bar">
                    <div class="capacity-fill level-safe" style="width: 0%;">
                        0% - Start typing!
                    </div>
                </div>
            </div>
            '''
            return

        # Create capacity bar
        usage_width = min(100, analysis.get('usage_percent', 0))
        level_class = f"level-{analysis.get('level', 'safe')}"

        capacity_html = f'''
        <div class="capacity-monitor">
            <h4>📊 Real-time Military Capacity Monitor</h4>
            <div class="capacity-bar">
                <div class="capacity-fill {level_class}" style="width: {usage_width}%;">
                    {analysis.get('usage_percent', 0):.1f}% Used
                </div>
            </div>
            <div style="margin: 15px 0; font-weight: bold;">
                {analysis.get('message', 'Analyzing...')}
            </div>
            <div class="feature-grid">
                <div class="feature-card">
                    <strong>Original:</strong> {analysis.get('original_size', 0):,} bytes
                </div>
                <div class="feature-card">
                    <strong>Estimated:</strong> {analysis.get('estimated_size', 0):,} bytes
                </div>
                <div class="feature-card">
                    <strong>Remaining:</strong> {analysis.get('bytes_remaining', 0):,} bytes
                </div>
                <div class="feature-card">
                    <strong>Compression:</strong> {'Yes' if analysis.get('compression_used', False) else 'No'}
                </div>
            </div>
        </div>
        '''

        self.capacity_display.value = capacity_html

    def _update_message_analysis(self, message: str, analysis: Dict[str, Any]):
        """Update detailed message analysis"""
        if not message.strip():
            self.message_analysis.value = ''
            return

        try:
            # Character analysis
            char_count = len(message)
            bengali_chars = sum(1 for c in message if '\u0980' <= c <= '\u09FF')
            ascii_chars = sum(1 for c in message if ord(c) < 128)
            other_chars = char_count - bengali_chars - ascii_chars

            analysis_html = f'''
            <div class="smart-card">
                <h4>📝 Military Message Analysis</h4>
                <div class="feature-grid">
                    <div class="feature-card">
                        <strong>📊 Characters</strong><br>{char_count:,} total
                    </div>
                    <div class="feature-card">
                        <strong>💾 UTF-8 Size</strong><br>{analysis.get('original_size', 0):,} bytes
                    </div>
                    <div class="feature-card">
                        <strong>🔐 Encrypted</strong><br>~{analysis.get('estimated_size', 0):,} bytes
                    </div>
                    <div class="feature-card">
                        <strong>📈 Usage</strong><br>{analysis.get('usage_percent', 0):.1f}%
                    </div>
                </div>

                <h5>🌐 Character Composition:</h5>
                <div style="font-family: monospace;">
                    • Bengali: {bengali_chars:,} ({bengali_chars/char_count*100:.1f}%)<br>
                    • ASCII: {ascii_chars:,} ({ascii_chars/char_count*100:.1f}%)<br>
                    • Other Unicode: {other_chars:,} ({other_chars/char_count*100:.1f}%)
                </div>
            '''

            if analysis.get('compression_used', False):
                analysis_html += f'''
                <div style="background: #e8f5e9; padding: 10px; border-radius: 8px; margin-top: 10px;">
                    <strong>🤖 Military compression will save ~{analysis.get('compression_savings', 0):,} bytes</strong>
                </div>
                '''

            analysis_html += '</div>'
            self.message_analysis.value = analysis_html

        except Exception as e:
            self.message_analysis.value = f'<div class="smart-card" style="border-left-color: #dc3545;">Analysis error: {e}</div>'

    def _on_options_change(self, change):
        """Handle smart options changes"""
        if hasattr(self, 'message') and self.message.value:
            self._on_message_change({'new': self.message.value})

    def _update_encoding_state(self):
        """Update encoding button state with colorful styling"""
        image_ready = hasattr(self, 'cover_image')
        password_ready = (len(self.password.value) >= 8 and
                         self.password.value == self.password_confirm.value)
        message_ready = bool(self.message.value.strip())

        if image_ready and password_ready and message_ready and self.capacity_monitor:
            analysis = self.capacity_monitor.analyze(self.message.value, getattr(self.auto_compress, 'value', True))

            if analysis['can_fit']:
                self.encode_button.disabled = False
                self.encode_button.remove_class('btn-disabled')
                self.encode_button.add_class('btn-encode')
                self.encode_button.description = '🔐 Military Encrypt & Embed'
            else:
                self.encode_button.disabled = True
                self.encode_button.remove_class('btn-encode')
                self.encode_button.add_class('btn-disabled')
                self.encode_button.description = '❌ Message Too Large'
        else:
            self.encode_button.disabled = True
            self.encode_button.remove_class('btn-encode')
            self.encode_button.add_class('btn-disabled')

    def _perform_encoding(self, button):
        """Perform the complete encoding process"""
        with self.encode_output:
            clear_output()

            if not CRYPTO_AVAILABLE:
                print("❌ Cryptography library not available!")
                return

            print("🚀 Starting Military Grade Encoding Process...")
            print("=" * 50)

            try:
                start_time = time.time()

                # Get inputs
                message = self.message.value.strip()
                password = self.password.value.strip()
                auto_compress = self.auto_compress.value
                verify_integrity = self.sha256_verify.value

                print(f"🤖 Military Configuration:")
                print(f"   • Auto-compression: {'Enabled' if auto_compress else 'Disabled'}")
                print(f"   • SHA-256 verification: {'Enabled' if verify_integrity else 'Disabled'}")

                # Step 1: Military Encryption
                print(f"\n🔐 Step 1: Military AES-256-GCM Encryption...")
                message_bytes = message.encode('utf-8')

                encrypted_data = MilitaryAESGCM.encrypt(message_bytes, password, auto_compress)

                # Step 2: Advanced PVD Embedding
                print(f"\n🎨 Step 2: Advanced PVD Embedding...")

                self.encoded_image = self.pvd.embed(
                    self.cover_image,
                    encrypted_data,
                    progress_callback=self._update_progress
                )

                total_time = time.time() - start_time

                print(f"\n🎉 MILITARY ENCODING COMPLETED SUCCESSFULLY!")
                print(f"⏱️ Total time: {total_time:.3f}s")
                print("✅ Your message is now securely hidden with military-grade encryption!")

                # Enable download
                self.download_button.disabled = False
                self.download_button.remove_class('btn-disabled')
                self.download_button.add_class('btn-download')

                # Show preview
                print("\n🖼️ Image Comparison:")
                self._show_comparison()

            except Exception as e:
                print(f"❌ Military encoding failed: {e}")
                import traceback
                traceback.print_exc()

    def _update_progress(self, block_progress: float, bit_progress: float):
        """Update encoding progress"""
        progress_html = f'''
        <div class="capacity-monitor">
            <div class="capacity-bar">
                <div class="capacity-fill level-moderate" style="width: {bit_progress:.1f}%;">
                    {bit_progress:.1f}% Complete
                </div>
            </div>
            <div style="font-family: monospace; margin-top: 10px;">
                🔧 Block Processing: {block_progress:.1f}%<br>
                🔐 Data Embedding: {bit_progress:.1f}%
            </div>
        </div>
        '''
        self.encode_progress.value = progress_html

    def _show_comparison(self):
        """Show before/after image comparison"""
        try:
            print("Original Cover Image:")
            orig_thumb = self.cover_image.copy()
            orig_thumb.thumbnail((400, 400))
            display(orig_thumb)

            print("\nMilitary Encoded Image (with hidden message):")
            encoded_thumb = self.encoded_image.copy()
            encoded_thumb.thumbnail((400, 400))
            display(encoded_thumb)

            print("👁️ Visual difference should be imperceptible!")

        except Exception as e:
            print(f"Display error: {e}")

    def _download_image(self, button):
        """Download the encoded image"""
        with self.encode_output:
            try:
                if not hasattr(self, 'encoded_image'):
                    print("❌ No encoded image available!")
                    return

                print("💾 Preparing military-grade download...")

                # Convert to optimized PNG
                img_buffer = io.BytesIO()
                self.encoded_image.save(
                    img_buffer,
                    format='PNG',
                    optimize=False,  # Don't optimize to preserve data
                    compress_level=1
                )
                img_data = img_buffer.getvalue()

                b64_data = base64.b64encode(img_data).decode()
                timestamp = int(time.time())
                filename = f"military_bengali_steganography_{timestamp}.png"

                download_html = f'''
                <div class="smart-card success-animation" style="text-align: center;">
                    <h3>💾 Military Download Ready!</h3>
                    <p>Your secret message is now safely hidden with military-grade optimization.</p>

                    <a href="data:image/png;base64,{b64_data}"
                       download="{filename}"
                       style="display: inline-block; background: linear-gradient(135deg, #11998e, #38ef7d);
                              color: white; padding: 25px 50px; text-decoration: none;
                              border-radius: 15px; margin: 20px 0; font-weight: bold;
                              font-size: 20px; box-shadow: 0 8px 25px rgba(17,153,142,0.3);
                              transition: transform 0.3s ease;">
                       📥 Download Military Image
                    </a>

                    <div style="margin-top: 20px; font-size: 14px; color: #666;">
                        <strong>📋 Military File Details:</strong><br>
                        • Format: PNG (lossless)<br>
                        • Size: {len(img_data):,} bytes<br>
                        • Security: Military AES-256-GCM + Advanced PVD<br>
                        • Features: Auto-compression + SHA-256 integrity<br>
                        • Filename: {filename}
                    </div>

                    <div style="background: #fff3cd; padding: 15px; border-radius: 10px; margin-top: 20px;">
                        <strong>⚠️ Important Military Security Notes:</strong><br>
                        • Keep the original cover image safe (required for decoding)<br>
                        • Remember your encryption password<br>
                        • Share only with intended recipients<br>
                        • This image contains your military-encrypted secret message
                    </div>
                </div>
                '''

                clear_output()
                display(HTML(download_html))
                print("✅ Military encoding process completed successfully!")

            except Exception as e:
                print(f"❌ Download error: {e}")

    # Decoding Event Handlers
    def _on_decode_uploads(self, change):
        """Handle decode image uploads"""
        if change['new']:
            try:
                uploaded_file = list(change['new'].values())[0]

                if change['owner'] == self.cover_upload:
                    self.decode_cover_image = Image.open(io.BytesIO(uploaded_file['content'])).convert('RGB')
                    print("✅ Cover image loaded")
                else:
                    self.decode_stego_image = Image.open(io.BytesIO(uploaded_file['content'])).convert('RGB')
                    print("✅ Encoded image loaded")

                self._update_decode_info()

            except Exception as e:
                self.decode_info.value = f'''
                <div class="smart-card" style="border-left-color: #dc3545;">
                    <strong>❌ Image loading error:</strong> {e}
                </div>
                '''

    def _on_decode_password(self, change):
        """Handle decode password input"""
        self._update_decode_info()

    def _update_decode_info(self):
        """Update decode information display"""
        has_cover = hasattr(self, 'decode_cover_image')
        has_stego = hasattr(self, 'decode_stego_image')
        has_password = bool(self.decode_password.value.strip())

        status_parts = []

        if has_cover and has_stego:
            if self.decode_cover_image.size == self.decode_stego_image.size:
                status_parts.append("✅ Image dimensions match perfectly")

                # Analyze differences if possible
                try:
                    # Simple pixel difference analysis
                    cover_pixels = list(self.decode_cover_image.getdata())
                    stego_pixels = list(self.decode_stego_image.getdata())

                    changed_pixels = sum(1 for c, s in zip(cover_pixels, stego_pixels) if c != s)
                    total_pixels = len(cover_pixels)

                    if changed_pixels > 0:
                        status_parts.append(f"📊 Modified pixels: {changed_pixels:,}/{total_pixels:,} ({changed_pixels/total_pixels*100:.3f}%)")
                        status_parts.append("🎯 Hidden data detected!")
                    else:
                        status_parts.append("⚠️ No differences found")

                except Exception:
                    status_parts.append("📊 Could not analyze differences")
            else:
                status_parts.append("❌ Image dimensions don't match!")

        if has_password:
            if len(self.decode_password.value) >= 8:
                status_parts.append("✅ Password provided")
            else:
                status_parts.append("⚠️ Password seems short")

        # Update button state
        ready = (has_cover and has_stego and has_password and
                hasattr(self, 'decode_cover_image') and hasattr(self, 'decode_stego_image') and
                self.decode_cover_image.size == self.decode_stego_image.size)

        self.decode_button.disabled = not ready

        if ready:
            self.decode_button.remove_class('btn-disabled')
            self.decode_button.add_class('btn-decode')
            overall_status = "🚀 Ready for military decoding!"
            status_color = "#28a745"
        else:
            self.decode_button.remove_class('btn-decode')
            self.decode_button.add_class('btn-disabled')
            overall_status = "⏳ Provide all required components"
            status_color = "#ffc107"

        info_html = f'''
        <div class="smart-card" style="border-left-color: {status_color};">
            <h4 style="color: {status_color};">{overall_status}</h4>
            <ul>
                {"".join(f"<li>{part}</li>" for part in status_parts)}
            </ul>

            <div style="background: #e8f5e9; padding: 15px; border-radius: 8px; margin-top: 15px;">
                <strong>🤖 Military Features:</strong><br>
                • Automatic compression detection<br>
                • SHA-256 integrity verification<br>
                • Enhanced error handling<br>
                • Real-time processing feedback
            </div>
        </div>
        '''

        self.decode_info.value = info_html

    def _perform_decoding(self, button):
        """Perform the complete decoding process"""
        with self.decode_output:
            clear_output()

            if not CRYPTO_AVAILABLE:
                print("❌ Cryptography library not available!")
                return

            print("🔍 Starting Military Grade Decoding Process...")
            print("=" * 50)

            try:
                start_time = time.time()
                password = self.decode_password.value.strip()
                verify_integrity = self.verify_integrity.value
                detailed_analysis = self.detailed_analysis.value

                print(f"🤖 Military Configuration:")
                print(f"   • SHA-256 verification: {'Enabled' if verify_integrity else 'Disabled'}")
                print(f"   • Detailed analysis: {'Enabled' if detailed_analysis else 'Disabled'}")

                # Step 1: Advanced PVD Extraction
                print(f"\n🎨 Step 1: Advanced PVD Data Extraction...")

                encrypted_data = self.pvd.extract(
                    self.decode_cover_image,
                    self.decode_stego_image,
                    progress_callback=self._update_decode_progress
                )

                if not encrypted_data:
                    print("❌ No hidden data found!")
                    self._show_no_data_error()
                    return

                print(f"✅ Extracted {len(encrypted_data):,} bytes of encrypted data")

                # Analyze package if detailed mode
                if detailed_analysis:
                    self._analyze_package(encrypted_data)

                # Step 2: Military Decryption
                print(f"\n🔐 Step 2: Military AES-256-GCM Decryption...")

                decrypted_bytes = MilitaryAESGCM.decrypt(
                    encrypted_data,
                    password,
                    verify_integrity=verify_integrity
                )
                decrypted_message = decrypted_bytes.decode('utf-8')

                total_time = time.time() - start_time

                print(f"\n🎉 MILITARY DECODING COMPLETED SUCCESSFULLY!")
                print(f"⏱️ Total time: {total_time:.3f}s")

                # Display the message
                self._display_decoded_message(decrypted_message, total_time)

            except ValueError as e:
                error_msg = str(e).lower()
                if 'password' in error_msg or 'decrypt' in error_msg:
                    print("🔐 Decryption failed: Incorrect password")
                    self._show_password_error()
                elif 'integrity' in error_msg:
                    print("🔍 SHA-256 integrity verification failed")
                    self._show_integrity_error()
                else:
                    print(f"❌ Decryption error: {e}")
                    self._show_general_error(str(e))

            except Exception as e:
                print(f"❌ Military decoding failed: {e}")
                self._show_general_error(str(e))

    def _update_decode_progress(self, block_progress: float, extraction_progress: float):
        """Update decoding progress"""
        progress_html = f'''
        <div class="capacity-monitor">
            <div class="capacity-bar">
                <div class="capacity-fill level-moderate" style="width: {extraction_progress:.1f}%;">
                    {extraction_progress:.1f}% Extracted
                </div>
            </div>
            <div style="font-family: monospace; margin-top: 10px;">
                🔍 Block Analysis: {block_progress:.1f}%<br>
                📤 Data Extraction: {extraction_progress:.1f}%
            </div>
        </div>
        '''
        self.decode_progress.value = progress_html

    def _analyze_package(self, encrypted_data: bytes):
        """Analyze the encrypted package"""
        try:
            json_str = encrypted_data.decode('utf-8')
            package = json.loads(json_str)

            print(f"📦 Military Package Analysis:")
            print(f"   • Version: {package.get('version', 'unknown')}")
            print(f"   • Algorithm: {package.get('algorithm', 'unknown')}")
            print(f"   • Compressed: {'Yes' if package.get('compressed') else 'No'}")
            print(f"   • SHA-256 hash: {'Yes' if package.get('message_hash') else 'No'}")

            if package.get('compression_info'):
                comp_info = package['compression_info']
                print(f"   • Compression savings: {comp_info.get('savings', 0):.1f}%")

        except Exception as e:
            print(f"⚠️ Package analysis failed: {e}")

    def _display_decoded_message(self, message: str, process_time: float):
        """Display the decoded message with analysis"""
        char_count = len(message)
        byte_count = len(message.encode('utf-8'))
        bengali_chars = sum(1 for c in message if '\u0980' <= c <= '\u09FF')

        message_html = f'''
        <div class="smart-section success-animation">
            <h3>🔓 Military Decoded Secret Message</h3>

            <div style="font-family: 'SolaimanLipi', 'Kalpurush', 'Bangla', 'Noto Sans Bengali', serif;
                        font-size: 20px; line-height: 2; background: white; padding: 25px;
                        border-radius: 15px; border: 3px solid #28a745; margin: 20px 0;
                        box-shadow: 0 5px 15px rgba(40, 167, 69, 0.2);">
                {message}
            </div>

            <div class="feature-grid">
                <div class="feature-card">
                    <strong>📝 Characters</strong><br>
                    {char_count:,} total
                </div>
                <div class="feature-card">
                    <strong>💾 Size</strong><br>
                    {byte_count:,} bytes
                </div>
                <div class="feature-card">
                    <strong>🌐 Bengali</strong><br>
                    {bengali_chars:,} chars ({bengali_chars/char_count*100:.1f}%)
                </div>
                <div class="feature-card">
                    <strong>⏱️ Process Time</strong><br>
                    {process_time:.3f}s
                </div>
            </div>

            <div style="background: #d4edda; padding: 20px; border-radius: 10px; margin-top: 20px;">
                <strong>✅ Military Verification Complete:</strong><br>
                • Military AES-256-GCM decryption: ✅ Success<br>
                • Auto-decompression: ✅ Applied if needed<br>
                • SHA-256 integrity: ✅ {'Verified' if self.verify_integrity.value else 'Skipped'}<br>
                • UTF-8 encoding: ✅ Valid<br>
                • Message extraction: ✅ Complete
            </div>
        </div>
        '''

        self.decoded_message.value = message_html

    def _show_no_data_error(self):
        """Show no data found error"""
        self.decoded_message.value = '''
        <div class="smart-card" style="border-left-color: #ffc107;">
            <h3>⚠️ No Hidden Data Found</h3>
            <p><strong>Possible reasons:</strong></p>
            <ul>
                <li>Images don't contain steganographic data</li>
                <li>Wrong cover or encoded image provided</li>
                <li>Images were compressed after encoding</li>
                <li>Different steganography tool was used</li>
            </ul>
        </div>
        '''

    def _show_password_error(self):
        """Show password error"""
        self.decoded_message.value = '''
        <div class="smart-card" style="border-left-color: #dc3545;">
            <h3>🔐 Military Decryption Failed</h3>
            <p><strong>Most likely cause:</strong> Incorrect password</p>
            <p><strong>Other possibilities:</strong></p>
            <ul>
                <li>Data corrupted during extraction</li>
                <li>Wrong encoded image provided</li>
                <li>Images modified after encoding</li>
            </ul>
            <p><strong>Solution:</strong> Verify your password and try again</p>
        </div>
        '''

    def _show_integrity_error(self):
        """Show integrity verification error"""
        self.decoded_message.value = '''
        <div class="smart-card" style="border-left-color: #fd7e14;">
            <h3>🔍 Military Integrity Verification Failed</h3>
            <p><strong>Cause:</strong> SHA-256 message integrity check failed</p>
            <p><strong>This means:</strong></p>
            <ul>
                <li>Message may be corrupted</li>
                <li>Data was modified during transmission</li>
                <li>Extraction errors occurred</li>
            </ul>
            <p><strong>Recommendation:</strong> Re-check your source images</p>
        </div>
        '''

    def _show_general_error(self, error_msg: str):
        """Show general error"""
        self.decoded_message.value = f'''
        <div class="smart-card" style="border-left-color: #dc3545;">
            <h3>❌ Military Decoding Error</h3>
            <p><strong>Error:</strong> {error_msg}</p>
            <p><strong>Please verify:</strong></p>
            <ul>
                <li>Both images are provided correctly</li>
                <li>Images have identical dimensions</li>
                <li>Correct password is used</li>
                <li>Images haven't been modified</li>
            </ul>
        </div>
        '''

# ============================================================================
# MAIN APPLICATION INITIALIZATION
# ============================================================================

def initialize_military_application():
    """Initialize and launch the Advanced Bengali Steganography Suite"""
    # Initialize and display the GUI
    gui = AdvancedBengaliSteganographyGUI()
    return gui

# Entry point
if __name__ == "__main__":
    app = initialize_military_application()